In [1]:
import numpy as np
import pandas as pd
import math
import random
import pickle
import json

# to make the experimens replicable
random.seed(123456)

In [2]:
df_uffici = pd.read_csv('../../data/dg_struttura.csv', dtype=str)
id_label_dict = dict(zip(df_uffici['ID'].values, df_uffici['NOME'].values))

In [3]:
df = pd.read_pickle('../../data/atti_processed.pkl')

In [4]:
df.head()

,CODICE_PRATICA,DATA_ATTO,DESCRIZIONE,DESCRIZIONE_TIPO_ATTO,DESCRIZIONE_TIPO_PRATICA,ID_TIPO_PRATICA,LINK_CONTENUTO_BINARIO,OGGETTO,PERSONA,STATUS,UFFICIO_AC,UFFICIO_DG,UFFICIO_SE,publisher
0,20000000011999AD00000005121,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIP. ALESSANDRO ANNUNZIATI (MATR. 14117). ASPE...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows
1,20000000021999AD00000005123,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIP. GIOVANNI ORLANDINI (MATR. 14311). ASPETTA...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows
2,20000000031999AD00000005018,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIP. LEONELLO TOCCAFONDI. CONGEDO STRAORDINARI...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows
3,20000000041999AD00000005116,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIPENDENTI REGIONALI DIVERSI. CONGEDO STRAORDI...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows
4,20000000051999AD00000005127,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIP. SILVIA LORENZI (MATR. 13366). CONGEDO STR...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows


In [5]:
df.shape

(156189, 14)

In [6]:
# df = df[df['DATA_ATTO'].dt.year > 2007]
# df.shape

### Documents per Office

In [7]:
grouped = df.groupby(['UFFICIO_DG']).count()['CODICE_PRATICA'].sort_values(ascending=False)
grouped

UFFICIO_DG
50073    13825
50117    13592
50004    11820
50115    11590
50003     9724
50116     9448
50000     8290
50202     8095
50005     7479
01937     7160
50006     6021
50119     5712
01934     4923
50125     4655
01943     3833
50007     3431
01946     3347
01025     3030
01928     2710
50002     2624
50123     1850
50201     1663
01931     1582
02090     1564
01027     1444
50127     1410
01923     1085
99999      790
50112      780
50124      737
50122      465
50044      464
50114      457
50113      368
50200      116
50026      105
Name: CODICE_PRATICA, dtype: int64

we remove the observations for the following offices

In [8]:
offices = grouped.index[-8:]
offices

Index(['50112', '50124', '50122', '50044', '50114', '50113', '50200', '50026'], dtype='object', name='UFFICIO_DG')

In [9]:
for o in offices:
    df = df[df['UFFICIO_DG'] != o]

In [13]:
df.shape

(152697, 14)

## Dataset Creation

Indipendent Variable:
- 'OGGETTO'
- 'UFFICIO_DG'
- 'DESCRIZIONE_TIPO_ATTO'
- 'DESCRIZIONE_TIPO_PRATICA'
- 'publisher'

In [14]:
df_dataset = df[['OGGETTO', 'UFFICIO_DG']]

In [15]:
df_dataset.shape

(152697, 2)

In [16]:
samples = np.array([x[0] for x in df_dataset.as_matrix()])
labels = np.array([x[1] for x in df_dataset.as_matrix()])

/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [17]:
print(samples[10])
print(labels[10])

ELENCO REGIONALE DELLE AZIENDE PRODUTTRICI E FORNITRICI DI PROTESI ED AUSILI DI CUI AL DECRETO N. 1317 DEL 20.03.1998: MODIFICHE ED INTEGRAZIONI
01943


In [18]:
labels = np.array([id_label_dict[x] if x in id_label_dict else 'office_{}'.format(x) for x in labels])

In [19]:
indexes = np.array(range(len(samples)))
np.random.shuffle(indexes)

In [20]:
samples = samples[indexes]
labels = labels[indexes]

### Transform labels to ids

In [21]:
ids_labels = dict(enumerate(sorted(list(set(labels)))))
labels_ids = {v:k for k,v in ids_labels.items()}

In [22]:
labels_ids

{'ALTRI UFFICI': 0,
 'AVVOCATURA REGIONALE                                  ': 1,
 'D.G.  AVVOCATURA                                      ': 2,
 "D.G. COMPETITIVITA' DEL SISTEMA REGIONALE E SVILUPPO D": 3,
 'D.G. PRESIDENZA                                       ': 4,
 'DIPARTIMENTO BILANCIO E FINANZE                       ': 5,
 'DIPARTIMENTO ORGANIZZAZIONE                           ': 6,
 'DIPARTIMENTO ORGANIZZAZIONE E RISORSE                 ': 7,
 'DIPARTIMENTO POLITICHE FORMATIVE E BENI CULTURALI     ': 8,
 'DIPARTIMENTO POLITICHE TERRITORIALI E AMBIENTALI      ': 9,
 'DIPARTIMENTO PRESIDENZA AFFARI LEGISLATIVI E GIURIDICI': 10,
 'DIPARTIMENTO SALUTE E POLITICHE SOLIDARIETA           ': 11,
 'DIPARTIMENTO SVILUPPO ECONOMICO                       ': 12,
 'DIREZIONE AGRICOLTURA E SVILUPPO RURALE': 13,
 "DIREZIONE ATTIVITA' PRODUTTIVE": 14,
 'DIREZIONE DIFESA DEL SUOLO E PROTEZIONE CIVILE': 15,
 'DIREZIONE DIRITTI DI CITTADINANZA E COESIONE SOCIALE': 16,
 'DIREZIONE GENERALE BILANCIO 

In [23]:
generale = []
others = []
for k,_ in labels_ids.items():
        if 'D.G' in k:
            generale.append(k)
        elif 'GENERALE' in k:
            generale.append(k)
        else:
            others.append(k)
        

In [24]:
generale

['D.G.  AVVOCATURA                                      ',
 "D.G. COMPETITIVITA' DEL SISTEMA REGIONALE E SVILUPPO D",
 'D.G. PRESIDENZA                                       ',
 'DIREZIONE GENERALE BILANCIO E FINANZE                 ',
 'DIREZIONE GENERALE DIRITTO ALLA SALUTE E POLITICHE DI ',
 'DIREZIONE GENERALE POLITICHE FORMATIVE, BENI E ATTIVIT',
 "DIREZIONE GENERALE POLITICHE MOBILITA', INFRASTRUTTURE E TRASPORTO PUBBLICO LOCALE",
 'DIREZIONE GENERALE POLITICHE TERRITORIALI E AMBIENTALI',
 'DIREZIONE GENERALE SVILUPPO ECONOMICO                 ']

In [25]:
others

['ALTRI UFFICI',
 'AVVOCATURA REGIONALE                                  ',
 'DIPARTIMENTO BILANCIO E FINANZE                       ',
 'DIPARTIMENTO ORGANIZZAZIONE                           ',
 'DIPARTIMENTO ORGANIZZAZIONE E RISORSE                 ',
 'DIPARTIMENTO POLITICHE FORMATIVE E BENI CULTURALI     ',
 'DIPARTIMENTO POLITICHE TERRITORIALI E AMBIENTALI      ',
 'DIPARTIMENTO PRESIDENZA AFFARI LEGISLATIVI E GIURIDICI',
 'DIPARTIMENTO SALUTE E POLITICHE SOLIDARIETA           ',
 'DIPARTIMENTO SVILUPPO ECONOMICO                       ',
 'DIREZIONE AGRICOLTURA E SVILUPPO RURALE',
 "DIREZIONE ATTIVITA' PRODUTTIVE",
 'DIREZIONE DIFESA DEL SUOLO E PROTEZIONE CIVILE',
 'DIREZIONE DIRITTI DI CITTADINANZA E COESIONE SOCIALE',
 'DIREZIONE ISTRUZIONE E FORMAZIONE',
 'DIREZIONE LAVORO',
 'DIREZIONE ORGANIZZAZIONE E SISTEMI INFORMATIVI',
 'POLITICHE AMBIENTALI, ENERGIA E CAMBIAMENTI CLIMATICI',
 'UFFICI DEL GENIO CIVILE                               ']

In [27]:
with open('../../data/dataset/label_index.json', 'w') as f:
    json.dump(labels_ids,f)

In [28]:
encoded_labels = np.array([labels_ids[x] for x in labels])

### Transform the samples into ids

Removed numbers

In [29]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
nltk.download('punkt')

punctuation = ['-', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}', '’', '”', '“', '``', "''"]
stop_words = set(stopwords.words('italian'))
stop_words.update(punctuation)

[nltk_data] Downloading package stopwords to /home/fabio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/fabio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
def hasnumbers(value):
    return any(c.isdigit() for c in value)

In [31]:
def tokenize_sample(samples, remove_stopwords=True, tokenizer=word_tokenize):
    for sample in samples:
        words = []
        sample = sample.replace('`', ' ')
        sample = sample.replace("'", " ")
        sample = sample.replace("”", ' ')
        sample = sample.replace("“", ' ')
        for w in tokenizer(sample):
#             if not hasnumbers(w) and len(w) > 2:
            w = w.replace('_', '')
            if remove_stopwords:
                if w not in stop_words:
                    words.append(w.lower())
            else:
                words.append(w.lower())
        yield words

In [32]:
tokenizer = RegexpTokenizer(r'\w+')

In [33]:
samples_tokenized = list(tokenize_sample(samples, remove_stopwords=False))

In [34]:
from collections import Counter

counter = Counter()

for sentence in samples_tokenized:
    counter.update(sentence)

In [35]:
counter.most_common()[:30]

[('.', 351980),
 ('di', 249796),
 ('-', 120716),
 ('e', 112773),
 ('del', 101592),
 ('per', 86934),
 (',', 78607),
 ('n.', 55939),
 ('della', 54336),
 (')', 49919),
 ('(', 45638),
 ('a', 44254),
 ('``', 44048),
 ('in', 42986),
 ("''", 42172),
 ('dell', 35011),
 ('la', 31844),
 ('al', 30490),
 ('regionale', 28268),
 ('impegno', 27938),
 ('l.r', 25987),
 ('comune', 25948),
 ('delle', 24354),
 ('liquidazione', 23435),
 ('dei', 22934),
 ('alla', 22785),
 (':', 22587),
 ('art', 22412),
 ('ai', 21026),
 ('l', 20608)]

The words are enumerated in ascending order so that we can select the top k words during the classification task

In [36]:
counter.__len__()

127444

In [37]:
words = [w for w,c in counter.most_common() if c > 2]

In [38]:
words[:30]

['.',
 'di',
 '-',
 'e',
 'del',
 'per',
 ',',
 'n.',
 'della',
 ')',
 '(',
 'a',
 '``',
 'in',
 "''",
 'dell',
 'la',
 'al',
 'regionale',
 'impegno',
 'l.r',
 'comune',
 'delle',
 'liquidazione',
 'dei',
 'alla',
 ':',
 'art',
 'ai',
 'l']

In [39]:
len(words) + 3

34731

In [40]:
id_word_dict = dict(enumerate([w for w in words],3))
word_id_dict = {v:k for k,v in id_word_dict.items()}

In [41]:
with open('../../data/dataset/id_word_dict.json','w') as f:
    json.dump(id_word_dict,f)

In [42]:
pad_char = 0
start_char=1
oov_char=2

In [43]:
def to_sequece_of_ids(tokenized_samples, word_id_dict):
    results = []
    for sample in tokenized_samples:
        encoded_sample = []
        for w in sample:
            if w in word_id_dict:
                encoded_sample.append(word_id_dict[w])
            else:
                encoded_sample.append(oov_char)
        results.append(encoded_sample)
    return results

In [44]:
encoded_samples = np.array(to_sequece_of_ids(samples_tokenized, word_id_dict))

In [45]:
encoded_samples[:4]

array([list([108, 4, 466, 27, 426, 98, 117, 478, 4, 1933, 4624, 14, 58, 6003, 389, 63, 29, 643, 2351, 6, 2, 3, 333]),
       list([85, 2, 15, 147, 21, 25, 712, 17, 537, 116, 55, 234, 7, 524, 319, 17, 959, 4, 1161, 17, 3]),
       list([41, 2644, 22781, 3, 22, 4, 40, 14, 48, 4, 2, 528, 4, 212, 9, 8, 75, 56, 838, 2526, 235, 4, 308, 4, 65, 38, 38, 3, 74, 7, 161, 3, 10, 2, 7, 27201]),
       list([68, 989, 2352, 5, 41, 8938, 2352, 3, 22, 4, 73, 8, 690, 110, 292, 1583, 7, 205, 803, 20, 41])],
      dtype=object)

In [46]:
max(np.max(encoded_samples))

34661

## Create Train and Test Set

- shuffle data
- split the dataset into 80-20

In [47]:
from sklearn.model_selection import StratifiedShuffleSplit

In [48]:
split_train_test = StratifiedShuffleSplit(1,test_size=0.2, random_state=123456)

for train, test in split_train_test.split(encoded_samples, encoded_labels):
    train_samples, test_samples = encoded_samples[train], encoded_samples[test]
    train_labels, test_labels = encoded_labels[train], encoded_labels[test]

## Save the Dataset

In [49]:
np.savez_compressed('../../data/dataset/atti_dataset.npz',
                   x_train=train_samples, y_train=train_labels,
                   x_test=test_samples, y_test=test_labels
                   )